In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from tqdm import tqdm
import timm
import torchattacks
import matplotlib.pyplot as plt
import torchvision
import numpy as np
import detectors
import json

In [12]:
DATASET_SUBSET = False
DATASET_SUBSET_SIZE = 10000

In [13]:
norm_mean = (0.4914, 0.4822, 0.4465)
norm_std =  (0.2023, 0.1994, 0.201)

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std),
])

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

if DATASET_SUBSET:
    train_subset_indices = list(range(0, DATASET_SUBSET_SIZE))
    test_subset_indices = list(range(0, DATASET_SUBSET_SIZE // 2))
    trainset = torch.utils.data.Subset(trainset, train_subset_indices)
    testset = torch.utils.data.Subset(testset, test_subset_indices)

print(f'Train set size: {len(trainset)}')
print(f'Test set size: {len(testset)}')

Files already downloaded and verified
Files already downloaded and verified
Train set size: 50000
Test set size: 10000


In [14]:
batch_size = 128
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

print(f'Train loader batches: {len(trainloader)}')
print(f'Test loader batches: {len(testloader)}')

Train loader batches: 391
Test loader batches: 79


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

def get_pretrainded_model():
    return timm.create_model('resnet18_cifar10', pretrained=True).to(device)

cuda


In [16]:
def evaluate_attack(model, attack, dataloader):
    model.eval()
    correct = 0
    total = 0
    with tqdm(dataloader, unit="batch") as tepoch:
        tepoch.set_description(f"Evaluation")
        for inputs, labels in tepoch:
            inputs, labels = inputs.to(device), labels.to(device)

            if attack:
                adv_inputs = attack(inputs, labels)
            else:
                adv_inputs = inputs

            outputs = model(adv_inputs)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy


In [17]:
class AdversarialDataset(torch.utils.data.Dataset):
    def __init__(self, dataloader, model, attack):
        self.adversarial_images = []
        self.labels = []
        self.model = model
        
        model.eval()

        for inputs, labels in tqdm(dataloader, desc="Generating Adversarial Dataset"):
            inputs, labels = inputs.to(device), labels.to(device)

            adv_images = attack(inputs, labels)
        
            self.adversarial_images.extend(list(torch.unbind(adv_images.detach().cpu(), dim=0)))
            self.labels.extend(labels.detach().cpu().tolist())

    def __len__(self):
        return len(self.adversarial_images)

    def __getitem__(self, idx):
        return self.adversarial_images[idx], self.labels[idx]

In [18]:
def run_experiment(model, trainloader, testloader, attack, num_iters=5, num_epochs=20):
    results = {
        'pre_train_results': {
            'attack_test_acc': 0,
            'test_acc': 0,
        },
        'training_results': [],
    }
    
    attacktestset = AdversarialDataset(testloader, model, attack)
    attacktestloader = torch.utils.data.DataLoader(attacktestset, batch_size=batch_size, shuffle=True)
    
    attack_test_acc = evaluate_attack(model, None, attacktestloader)
    test_acc = evaluate_attack(model, None, testloader)
    print(f'Test: attack testset accuracy: {attack_test_acc:.2f}%')
    print(f'Test: testset accuracy: {test_acc:.2f}%')
    results['pre_train_results']['attack_test_acc'] = attack_test_acc
    results['pre_train_results']['test_acc'] = test_acc

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001,
                      momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    for i in range(num_iters):
        print(f'#### Iteration: {i} ####', flush=True)

        results['training_results'].append({'epochs_acc': [], 'attack_test_acc': 0, 'test_acc': 0, 'combined_acc': 0, 'oneshot_attack_test_acc': 0})

        attacktrainset = AdversarialDataset(trainloader, model, attack)

        combinedset = torch.utils.data.ConcatDataset([trainloader.dataset, attacktrainset])
        combinedloader = torch.utils.data.DataLoader(combinedset, batch_size=batch_size, shuffle=True)

        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0

            with tqdm(combinedloader, unit="batch") as tepoch:
                tepoch.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
                for inputs, labels in tepoch:
                    inputs, labels = inputs.to(device), labels.to(device)

                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

                    running_loss += loss.item()
                    tepoch.set_postfix(loss=loss.item())
            scheduler.step()

            epoch_loss = running_loss / len(combinedloader)
            epoch_acc = evaluate_attack(model, None, combinedloader)
            epoch_test_acc = evaluate_attack(model, None, testloader)
            print(f"Epoch [{epoch+1}/{num_epochs}]: Average Loss: {epoch_loss:.4f}")
            print(f"Epoch [{epoch+1}/{num_epochs}]: Train combinedset accuracy: {epoch_acc:.2f}%")
            print(f"Epoch [{epoch+1}/{num_epochs}]: Test: testset accuracy: {epoch_test_acc:.2f}%")
            results['training_results'][-1]['epochs_acc'].append(epoch_acc)

        oneshotattacktestset = AdversarialDataset(testloader, model, attack)
        oneshotattacktestloader = torch.utils.data.DataLoader(oneshotattacktestset, batch_size=batch_size, shuffle=True)

        attack_test_acc = evaluate_attack(model, None, attacktestloader)
        test_acc = evaluate_attack(model, None, testloader)
        combined_acc = evaluate_attack(model, None, combinedloader)
        oneshot_attack_test_acc = evaluate_attack(model, None, oneshotattacktestloader)
        print(f'Test: attack testset accuracy: {attack_test_acc:.2f}%')
        print(f'Test: testset accuracy: {test_acc:.2f}%')
        print(f'Train: combinedset accuracy: {combined_acc:.2f}%')
        print(f'Test: oneshot attack testset accuracy: {oneshot_attack_test_acc:.2f}%')
        results['training_results'][-1]['attack_test_acc'] = attack_test_acc
        results['training_results'][-1]['test_acc'] = test_acc
        results['training_results'][-1]['combined_acc'] = combined_acc
        results['training_results'][-1]['oneshot_attack_test_acc'] = oneshot_attack_test_acc
    return results

In [19]:
eps = 0.02

In [20]:
model = get_pretrainded_model()

fgsm = torchattacks.FGSM(model, eps=eps)
fgsm.set_normalization_used(mean=norm_mean, std=norm_std)

fgsm_results = run_experiment(model, trainloader, testloader, fgsm)

with open("fgsm_results.json", "w") as json_file:
    json.dump(fgsm_results, json_file, indent=4)

Evaluation: 100%|██████████| 79/79 [00:05<00:00, 15.05batch/s]

Test: attack testset accuracy: 30.06%
Test: testset accuracy: 94.98%
#### Iteration: 0 ####



Evaluation: 100%|██████████| 79/79 [00:04<00:00, 16.15batch/s]


Epoch [1/20]: Average Loss: 0.3827
Epoch [1/20]: Train combinedset accuracy: 93.56%
Epoch [1/20]: Test: testset accuracy: 90.81%


Evaluation: 100%|██████████| 79/79 [00:04<00:00, 16.21batch/s]


...

Epoch [19/20]: Average Loss: 0.0017
Epoch [19/20]: Train combinedset accuracy: 100.00%
Epoch [19/20]: Test: testset accuracy: 94.00%


Evaluation: 100%|██████████| 79/79 [00:04<00:00, 16.10batch/s]


Epoch [20/20]: Average Loss: 0.0017
Epoch [20/20]: Train combinedset accuracy: 99.99%
Epoch [20/20]: Test: testset accuracy: 94.18%


Evaluation: 100%|██████████| 79/79 [00:02<00:00, 27.04batch/s]

Test: attack testset accuracy: 68.53%
Test: testset accuracy: 94.18%
Train: combinedset accuracy: 100.00%
Test: oneshot attack testset accuracy: 36.43%


In [21]:
model = get_pretrainded_model()

bim = torchattacks.BIM(model, eps=eps, alpha=0.01, steps=10)
bim.set_normalization_used(mean=norm_mean, std=norm_std)

bim_results = run_experiment(model, trainloader, testloader, bim)

with open("bim_results.json", "w") as json_file:
    json.dump(bim_results, json_file, indent=4)

Evaluation: 100%|██████████| 79/79 [00:04<00:00, 16.10batch/s]

Test: attack testset accuracy: 0.04%
Test: testset accuracy: 94.98%
#### Iteration: 0 ####



Evaluation: 100%|██████████| 79/79 [00:04<00:00, 16.26batch/s]


Epoch [1/20]: Average Loss: 0.3808
Epoch [1/20]: Train combinedset accuracy: 95.19%
Epoch [1/20]: Test: testset accuracy: 91.24%


Evaluation: 100%|██████████| 79/79 [00:04<00:00, 16.20batch/s]


...

Epoch [19/20]: Average Loss: 0.0016
Epoch [19/20]: Train combinedset accuracy: 100.00%
Epoch [19/20]: Test: testset accuracy: 94.26%


Evaluation: 100%|██████████| 79/79 [00:05<00:00, 14.38batch/s]


Epoch [20/20]: Average Loss: 0.0016
Epoch [20/20]: Train combinedset accuracy: 100.00%
Epoch [20/20]: Test: testset accuracy: 94.30%


Evaluation: 100%|██████████| 79/79 [00:03<00:00, 23.69batch/s]

Test: attack testset accuracy: 60.83%
Test: testset accuracy: 94.30%
Train: combinedset accuracy: 100.00%
Test: oneshot attack testset accuracy: 0.73%


In [ ]:
model = get_pretrainded_model()

pgd = torchattacks.PGD(model, eps=eps, alpha=0.01, steps=40)
pgd.set_normalization_used(mean=norm_mean, std=norm_std)

pgd_results = run_experiment(model, trainloader, testloader, pgd)

with open("pgd_results.json", "w") as json_file:
    json.dump(pgd_results, json_file, indent=4)

Evaluation: 100%|██████████| 79/79 [00:05<00:00, 14.07batch/s]

Test: attack testset accuracy: 0.01%
Test: testset accuracy: 94.98%
#### Iteration: 0 ####



Evaluation: 100%|██████████| 79/79 [00:05<00:00, 14.25batch/s]


Epoch [1/20]: Average Loss: 0.3425
Epoch [1/20]: Train combinedset accuracy: 96.07%
Epoch [1/20]: Test: testset accuracy: 91.79%


Evaluation: 100%|██████████| 79/79 [00:05<00:00, 13.79batch/s]


...

Evaluation: 100%|██████████| 79/79 [00:05<00:00, 15.40batch/s]


Epoch [20/20]: Average Loss: 0.0035
Epoch [20/20]: Train combinedset accuracy: 99.99%
Epoch [20/20]: Test: testset accuracy: 93.60%


Evaluation: 100%|██████████| 79/79 [00:03<00:00, 25.23batch/s]

Test: attack testset accuracy: 69.93%
Test: testset accuracy: 93.60%
Train: combinedset accuracy: 99.99%
Test: oneshot attack testset accuracy: 0.01%
#### Iteration: 4 ####



Generating Adversarial Dataset:  72%|███████▏  | 283/391 [14:03<05:24,  3.01s/it]